In [1]:
import json
import requests
import pandas as pd
from PIL import Image 
from IPython.display import display
from IPython.display import Image
from IPython.core.display import HTML 

In [3]:
URL = "https://data.scryfall.io/default-cards/default-cards-20240117100432.json"
full_scryfall_df = pd.DataFrame(json.loads(requests.get(URL).text))
full_scryfall_df.head()

,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,tcgplayer_etched_id,flavor_name,attraction_lights,color_indicator,printed_type_line,printed_text,variation_of,life_modifier,hand_modifier,content_warning
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,0000a54c-a511-4925-92dc-01b937f9afad,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,[],NaN,NaN,98659.0,NaN,Spirit,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df = full_scryfall_df[['name',                       # the name of the card - not technically necessary but helpful for debugging
                       'mana_cost',                  # what type of mana the card costs to summon
                       'cmc',                        # how much mana the card costs
                       'type_line',                  # the type of the card (creature, sorcery, etc)
                       'oracle_text',                # what the card does
                       'power', 'toughness',         # the strength of the card if it's a creature
                       'colors', 'color_identity',   # more info on what type of mana the card costs
                       'keywords',                   # the keywords on the card (more on this later)
                       'image_uris',
                       'set', 'released_at',         # when the card was released
                       'rarity',                     # how much the card was printed
                       'games',                      # games tells if it is legal online or in paper (we exclude online-only cards)
                       'legalities']]                # which formats the card is legal in
df = df[df['games'].apply(lambda i: 'paper' in i)]
def legal(legalities):
    v = legalities.values()
    if len(set(v)) == 1 and "not_legal" in v:
        return False
    return True

df = df[df['legalities'].apply(legal)]
df = df[~df["type_line"].str.contains("Token", na=False)] # remove tokens
df

,name,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,image_uris,set,released_at,rarity,games,legalities
0,Fury Sliver,{5}{R},6.0,Creature — Sliver,All Sliver creatures have double strike.,3,3,[R],[R],[],{'small': 'https://cards.scryfall.io/small/fro...,tsp,2006-10-06,uncommon,"[paper, mtgo]","{'standard': 'not_legal', 'future': 'not_legal..."
1,Kor Outfitter,{W}{W},2.0,Creature — Kor Soldier,"When Kor Outfitter enters the battlefield, you...",2,2,[W],[W],[],{'small': 'https://cards.scryfall.io/small/fro...,zen,2009-10-02,common,"[paper, mtgo]","{'standard': 'not_legal', 'future': 'not_legal..."
3,Siren Lookout,{2}{U},3.0,Creature — Siren Pirate,Flying\nWhen Siren Lookout enters the battlefi...,1,2,[U],[U],"[Flying, Explore]",{'small': 'https://cards.scryfall.io/small/fro...,xln,2017-09-29,common,"[arena, paper, mtgo]","{'standard': 'not_legal', 'future': 'not_legal..."
4,Web,{G},1.0,Enchantment — Aura,Enchant creature (Target a creature as you cas...,NaN,NaN,[G],[G],[Enchant],{'small': 'https://cards.scryfall.io/small/fro...,3ed,1994-04-01,rare,[paper],"{'standard': 'not_legal', 'future': 'not_legal..."
5,Surge of Brilliance,{1}{U},2.0,Instant,Paradox — Draw a card for each spell you've ca...,NaN,NaN,[U],[U],"[Paradox, Foretell]",{'small': 'https://cards.scryfall.io/small/fro...,who,2023-10-13,uncommon,[paper],"{'standard': 'not_legal', 'future': 'not_legal..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90817,Boomerang,{U}{U},2.0,Instant,Return target permanent to its owner's hand.,NaN,NaN,[U],[U],[],{'small': 'https://cards.scryfall.io/small/fro...,5ed,1997-03-24,common,[paper],"{'standard': 'not_legal', 'future': 'not_legal..."
90819,Horned Troll,{2}{G},3.0,Creature — Troll,{G}: Regenerate Horned Troll.,2,2,[G],[G],[],{'small': 'https://cards.scryfall.io/small/fro...,8ed,2003-07-28,common,"[paper, mtgo]","{'standard': 'not_legal', 'future': 'not_legal..."
90820,Faerie Bladecrafter,{2}{B},3.0,Creature — Faerie Rogue,Flying\nWhenever one or more Faeries you contr...,2,2,[B],[B],[Flying],{'small': 'https://cards.scryfall.io/small/fro...,woc,2023-09-08,rare,[paper],"{'standard': 'not_legal', 'future': 'not_legal..."
90821,Exultant Skymarcher,{1}{W}{W},3.0,Creature — Vampire Soldier,Flying,2,3,[W],[W],[Flying],{'small': 'https://cards.scryfall.io/small/fro...,rix,2018-01-19,common,"[arena, paper, mtgo]","{'standard': 'not_legal', 'future': 'not_legal..."


In [66]:
df['paragraph1'] = ''
df['paragraph2'] = ''
df['paragraph3'] = ''
df['paragraph4'] = ''
df['paragraph5'] = ''
df['paragraph6'] = ''
for idx, row in df.iterrows():
    if not isinstance(row['oracle_text'], str):
        continue
    pgs = row['oracle_text'].replace(row['name'], 'CARDNAME').split("\n")
    prev = None
    for i in range(len(pgs)):
        if pgs[i].startswith("•"):
            pgs[i] = prev + pgs[i]
        else:
            prev = pgs[i]
    for i, pg in enumerate(pgs): 
        df.at[idx, f'paragraph{i}'] = pg
df

,name,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,...,games,legalities,paragraph1,paragraph2,paragraph3,paragraph4,paragraph5,paragraph6,paragraph0,paragraph7
0,Fury Sliver,{5}{R},6.0,Creature — Sliver,All Sliver creatures have double strike.,3,3,[R],[R],[],...,"[paper, mtgo]","{'standard': 'not_legal', 'future': 'not_legal...",,,,,,,All Sliver creatures have double strike.,nan
1,Kor Outfitter,{W}{W},2.0,Creature — Kor Soldier,"When Kor Outfitter enters the battlefield, you...",2,2,[W],[W],[],...,"[paper, mtgo]","{'standard': 'not_legal', 'future': 'not_legal...",,,,,,,"When CARDNAME enters the battlefield, you may ...",nan
3,Siren Lookout,{2}{U},3.0,Creature — Siren Pirate,Flying\nWhen Siren Lookout enters the battlefi...,1,2,[U],[U],"[Flying, Explore]",...,"[arena, paper, mtgo]","{'standard': 'not_legal', 'future': 'not_legal...","When CARDNAME enters the battlefield, it explo...",,,,,,Flying,nan
4,Web,{G},1.0,Enchantment — Aura,Enchant creature (Target a creature as you cas...,NaN,NaN,[G],[G],[Enchant],...,[paper],"{'standard': 'not_legal', 'future': 'not_legal...",Enchanted creature gets +0/+2 and has reach. (...,,,,,,Enchant creature (Target a creature as you cas...,nan
5,Surge of Brilliance,{1}{U},2.0,Instant,Paradox — Draw a card for each spell you've ca...,NaN,NaN,[U],[U],"[Paradox, Foretell]",...,[paper],"{'standard': 'not_legal', 'future': 'not_legal...","Foretell {1}{U} (During your turn, you may pay...",,,,,,Paradox — Draw a card for each spell you've ca...,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90817,Boomerang,{U}{U},2.0,Instant,Return target permanent to its owner's hand.,NaN,NaN,[U],[U],[],...,[paper],"{'standard': 'not_legal', 'future': 'not_legal...",,,,,,,Return target permanent to its owner's hand.,nan
90819,Horned Troll,{2}{G},3.0,Creature — Troll,{G}: Regenerate Horned Troll.,2,2,[G],[G],[],...,"[paper, mtgo]","{'standard': 'not_legal', 'future': 'not_legal...",,,,,,,{G}: Regenerate CARDNAME.,nan
90820,Faerie Bladecrafter,{2}{B},3.0,Creature — Faerie Rogue,Flying\nWhenever one or more Faeries you contr...,2,2,[B],[B],[Flying],...,[paper],"{'standard': 'not_legal', 'future': 'not_legal...",Whenever one or more Faeries you control deal ...,"When CARDNAME dies, each opponent loses X life...",,,,,Flying,nan
90821,Exultant Skymarcher,{1}{W}{W},3.0,Creature — Vampire Soldier,Flying,2,3,[W],[W],[Flying],...,"[arena, paper, mtgo]","{'standard': 'not_legal', 'future': 'not_legal...",,,,,,,Flying,nan


In [69]:
df['paragraph4'].value_counts()

paragraph4
                                                                        75918
Choose two —• Destroy all creatures with mana value 4 or greater.          15
LEVEL 4+                                                                   13
Choose two —• Draw a card.                                                 11
Equip {3}                                                                  11
                                                                        ...  
IV — Look at the top card of each player's library.                         1
LEVEL 12+                                                                   1
{2}{R}: Level 3                                                             1
Rasputin can't have more than seven dream counters on it.                   1
Whenever you cast an instant or sorcery spell, choose one —• Scry 1.        1
Name: count, Length: 128, dtype: int64

In [6]:
card = "Colossal Skyturtle"
row = df[df['name'] == card]
row

,name,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,image_uris,set,released_at,rarity,games,legalities
86618,Colossal Skyturtle,{4}{G}{G}{U},7.0,Enchantment Creature — Turtle,"Flying, ward {2}\nChannel — {2}{G}, Discard Co...",6,5,"[G, U]","[G, U]","[Flying, Ward, Channel]",{'small': 'https://cards.scryfall.io/small/fro...,neo,2022-02-18,uncommon,"[paper, mtgo, arena]","{'standard': 'legal', 'future': 'legal', 'hist..."


In [19]:
import Levenshtein as ls

In [12]:
def paragraph_similarity(c1, c2):
    ...

for pg in row.oracle_text.iloc[0].split("\n"):
    print(pg)

Flying, ward {2}
Channel — {2}{G}, Discard Colossal Skyturtle: Return target card from your graveyard to your hand.
Channel — {1}{U}, Discard Colossal Skyturtle: Return target creature to its owner's hand.
